In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, convolve

# Step 1: Build Gaussian Pyramid
def gaussian_pyramid(image, num_octaves, num_scales):
    """Create a Gaussian Pyramid for the input image."""
    gaussian_images = []
    sigma = 1.6  # Initial sigma value for the Gaussian blur
    k = np.sqrt(2)  # Constant multiplicative factor between scales
    
    for octave in range(num_octaves):
        octave_images = []
        for scale in range(num_scales):
            sigma_scaled = sigma * (k ** scale)
            blurred_image = gaussian_filter(image, sigma=sigma_scaled)
            octave_images.append(blurred_image)
        
        # Downsample image for the next octave
        image = cv2.resize(image, (image.shape[1] // 2, image.shape[0] // 2))
        gaussian_images.append(octave_images)
        
    return gaussian_images

In [2]:
# Step 2: Compute Difference of Gaussians (DoG)
def difference_of_gaussians(gaussian_images):
    """Compute the Difference of Gaussians from the Gaussian Pyramid."""
    dog_images = []
    for octave_images in gaussian_images:
        octave_dogs = []
        for i in range(1, len(octave_images)):
            dog = octave_images[i] - octave_images[i - 1]
            octave_dogs.append(dog)
        dog_images.append(octave_dogs)
    return dog_images

In [3]:
# Step 3: Detect Keypoints (Local Extrema in Scale-Space)
def detect_keypoints(dog_images):
    """Detect keypoints by searching for local extrema in the DoG images."""
    keypoints = []
    for octave_index, octave_dogs in enumerate(dog_images):
        for scale_index, dog in enumerate(octave_dogs[1:-1], start=1):
            for i in range(1, dog.shape[0] - 1):
                for j in range(1, dog.shape[1] - 1):
                    # Check if pixel is a local extrema (maximum or minimum)
                    pixel = dog[i, j]
                    neighbors = dog[i-1:i+2, j-1:j+2].flatten()
                    prev_scale = octave_dogs[scale_index-1][i-1:i+2, j-1:j+2].flatten()
                    next_scale = octave_dogs[scale_index+1][i-1:i+2, j-1:j+2].flatten()
                    neighbors = np.concatenate((neighbors, prev_scale, next_scale))
                    
                    if pixel == np.max(neighbors) or pixel == np.min(neighbors):
                        keypoints.append((i, j, octave_index, scale_index))
    return keypoints

In [4]:
# Step 4: Assign Orientation to Keypoints
def assign_orientations(image, keypoints):
    """Assign dominant orientation to each keypoint based on image gradients."""
    gradient_magnitudes = []
    orientations = []
    
    # Compute image gradients using Sobel filters
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
    
    grad_x = convolve(image, sobel_x)
    grad_y = convolve(image, sobel_y)
    
    # Compute gradient magnitude and orientation
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    angle = np.arctan2(grad_y, grad_x) * (180 / np.pi)  # Convert to degrees
    
    for kp in keypoints:
        i, j, octave, scale = kp
        
        # Extract a patch around the keypoint to compute dominant orientation
        patch_mag = magnitude[i-1:i+2, j-1:j+2]
        patch_angle = angle[i-1:i+2, j-1:j+2]
        
        # Dominant orientation is the most frequent angle in the patch
        hist, bin_edges = np.histogram(patch_angle, bins=36, range=(-180, 180), weights=patch_mag)
        dominant_orientation = bin_edges[np.argmax(hist)]
        
        orientations.append((i, j, octave, scale, dominant_orientation))
    
    return orientations

In [ ]:
# Load image and convert to grayscale
image = cv2.imread('image_path.jpg', cv2.IMREAD_GRAYSCALE)

# Parameters
num_octaves = 4
num_scales = 5

# Step 1: Generate Gaussian Pyramid
gaussian_images = gaussian_pyramid(image, num_octaves, num_scales)

# Step 2: Compute Difference of Gaussians (DoG)
dog_images = difference_of_gaussians(gaussian_images)

# Step 3: Detect keypoints (local extrema)
keypoints = detect_keypoints(dog_images)

# Step 4: Assign orientations to keypoints
scale_invariant_keypoints = assign_orientations(image, keypoints)

# Visualize Keypoints
keypoint_image = image.copy()
for kp in scale_invariant_keypoints:
    i, j, _, _, orientation = kp
    cv2.circle(keypoint_image, (j, i), 3, (255, 0, 0), 1)

plt.figure(figsize=(10, 10))
plt.imshow(keypoint_image, cmap='gray')
plt.title('Scale-Invariant Keypoints')
plt.axis('off')
plt.show()

# Display a few keypoints
for i, kp in enumerate(scale_invariant_keypoints[:5]):
    print(f"Keypoint {i+1}: Location=({kp[1]}, {kp[0]}), Octave={kp[2]}, Scale={kp[3]}, Orientation={kp[4]:.2f}°")